In [ ]:
# install dependencies: (use cu111 because colab has CUDA 11.1)
%pip install torch==1.10.0+cu111 torchvision==0.11.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
%pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html

# install mmdet for inference demo
%pip install mmdet

# install mmpose dependencies
%pip install -r requirements.txt

# install mmpose in develop mode
%pip install -e .

# Check Pytorch installation
import torch, torchvision

# print('torch version:', torch.__version__, torch.cuda.is_available())
# print('torchvision version:', torchvision.__version__)

# # Check MMPose installation
# import mmpose

# print('mmpose version:', mmpose.__version__)

# Check mmcv installation
# from mmcv.ops import get_compiling_cuda_version, get_compiler_version

# print('cuda version:', get_compiling_cuda_version())
# print('compiler information:', get_compiler_version())

In [1]:
from mmcv import Config
import mmcv
import mmpose
import torch, torchvision

cfg = Config.fromfile('configs/animal/2d_kpt_sview_rgb_img/topdown_heatmap/acino/hrnet_w32_acino_256x256.py'
    #'./configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/hrnet_w32_coco_256x192.py'
)
# cfg.data_root = 'data'
# cfg.work_dir = 'work_dirs/hrnet_w32_acino_256x256'
# cfg.data['train']['ann_file']=f'{cfg.data_root}/acino/annotations/acino_train.json'
# cfg.data['train']['img_prefix']=f'{cfg.data_root}/acino/data/'
# cfg.data['test']['ann_file']=f'{cfg.data_root}/acino/annotations/acino_test.json'
# cfg.data['test']['img_prefix']=f'{cfg.data_root}/acino/data/'
# cfg.data['val']['ann_file']=f'{cfg.data_root}/acino/annotations/acino_val.json'
# cfg.data['val']['img_prefix']=f'{cfg.data_root}/acino/data/'
# cfg.gpu_ids = range(1)
# cfg.seed = 0
# cfg.total_epochs=50

In [ ]:
print(cfg.pretty_text)

In [ ]:
from mmpose.datasets import build_dataset
from mmpose.models import build_posenet
from mmpose.apis import train_model

# build dataset
datasets = [build_dataset(cfg.data.train)]

# build model
model = build_posenet(cfg.model)

# create work_dir
mmcv.mkdir_or_exist(cfg.work_dir)

# train model
train_model(
    model, datasets, cfg, distributed=False, validate=True, meta=dict())

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!
=> num_images: 5346
=> load 5289 samples
load checkpoint from http path: https://download.openmmlab.com/mmpose/pretrain_models/hrnet_w32-36af842e.pth


2022-09-11 17:43:27,719 - mmpose - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: head.0.0.0.conv1.weight, head.0.0.0.bn1.weight, head.0.0.0.bn1.bias, head.0.0.0.bn1.running_mean, head.0.0.0.bn1.running_var, head.0.0.0.bn1.num_batches_tracked, head.0.0.0.conv2.weight, head.0.0.0.bn2.weight, head.0.0.0.bn2.bias, head.0.0.0.bn2.running_mean, head.0.0.0.bn2.running_var, head.0.0.0.bn2.num_batches_tracked, head.0.0.0.conv3.weight, head.0.0.0.bn3.weight, head.0.0.0.bn3.bias, head.0.0.0.bn3.running_mean, head.0.0.0.bn3.running_var, head.0.0.0.bn3.num_batches_tracked, head.0.0.0.downsample.0.weight, head.0.0.0.downsample.1.weight, head.0.0.0.downsample.1.bias, head.0.0.0.downsample.1.running_mean, head.0.0.0.downsample.1.running_var, head.0.0.0.downsample.1.num_batches_tracked, head.0.1.0.conv1.weight, head.0.1.0.bn1.weight, head.0.1.0.bn1.bias, head.0.1.0.bn1.running_mean, head.0.1.0.bn1.running_var, head.0.1.0.bn1.num_batches_tracked, he

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
=> num_images: 764
=> load 757 samples


2022-09-11 17:43:39,711 - mmpose - INFO - Epoch [1][1/83]	lr: 5.000e-07, eta: 16:12:38, time: 7.032, data_time: 5.766, memory: 12494, heatmap_loss: 0.0015, acc_pose: 0.0210, loss: 0.0015
2022-09-11 17:43:40,910 - mmpose - INFO - Epoch [1][2/83]	lr: 1.499e-06, eta: 9:29:09, time: 1.199, data_time: 0.005, memory: 12826, heatmap_loss: 0.0015, acc_pose: 0.0302, loss: 0.0015
2022-09-11 17:43:42,235 - mmpose - INFO - Epoch [1][3/83]	lr: 2.498e-06, eta: 7:20:27, time: 1.325, data_time: 0.003, memory: 12826, heatmap_loss: 0.0016, acc_pose: 0.0185, loss: 0.0016
2022-09-11 17:43:43,377 - mmpose - INFO - Epoch [1][4/83]	lr: 3.497e-06, eta: 6:09:46, time: 1.142, data_time: 0.005, memory: 12826, heatmap_loss: 0.0015, acc_pose: 0.0129, loss: 0.0015
2022-09-11 17:43:44,523 - mmpose - INFO - Epoch [1][5/83]	lr: 4.496e-06, eta: 5:27:28, time: 1.146, data_time: 0.004, memory: 12826, heatmap_loss: 0.0015, acc_pose: 0.0323, loss: 0.0015
2022-09-11 17:43:45,745 - mmpose - INFO - Epoch [1][6/83]	lr: 5.495e-

In [ ]:
from matplotlib import pprint
from mmpose.apis import (inference_top_down_pose_model, init_pose_model,
                         vis_pose_result, process_mmdet_results)
# from mmdet.apis import inference_detector, init_detector
from xtcocotools.coco import COCO
import os
# import cv2
local_runtime = False

# try:
#     from google.colab.patches import cv2_imshow  # for image visualization in colab
# except:
#     local_runtime = True
from mmpose.datasets import DatasetInfo

pose_checkpoint = 'work_dirs/hrnet_w32_acino_256x256/latest.pth'
json_file='tests/data/acino/testacino.json'
coco = COCO(json_file)
img_keys = list(coco.imgs.keys())

# initialize pose model
pose_model = init_pose_model(cfg, pose_checkpoint)

dataset = pose_model.cfg.data['test']['type']
dataset_info = pose_model.cfg.data['test'].get('dataset_info', None)
dataset_info = DatasetInfo(dataset_info)
return_heatmap=False
output_layer_names=None

for i in mmcv.track_iter_progress(range(len(img_keys))):
  # get bounding box annotations
  image_id = img_keys[i]
  #print("\nID:",image_id,"\n")
  image = coco.loadImgs(image_id)[0]
  #print("\nID:",image,"\n")
  image_name = os.path.join('tests/data/acino', image['file_name'])
  #print("\nID:",image_name,"\n")
  ann_ids = coco.getAnnIds(image_id)
  
  person_results = []
  for ann_id in ann_ids:
    person = {}
    ann = coco.anns[ann_id]
    
    # bbox format is 'xywh'
    person['bbox'] = ann['bbox']
    person_results.append(person)
    # print('\n')
    # print(person_results)
    # print('\n')

  pose_results, returned_outputs = inference_top_down_pose_model(
    pose_model,
    image_name,
    person_results=person_results,##
    bbox_thr=None,
    format='xywh',
    dataset=dataset,
    dataset_info=dataset_info,
    return_heatmap=return_heatmap,
    outputs=output_layer_names)
  
  os.makedirs('vis_result', exist_ok=True)
  out_file = os.path.join('vis_result', f'vis_as{image_id}.jpg')
  vis_result=vis_pose_result(pose_model,image_name,pose_results,radius=6,thickness=3,out_file=out_file)
  